# Assignment 1: Dino Fun World

You, in your role as a burgeoning data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a couple questions about their park operations. In order to perform the requested analysis, they have provided you with a database containing information about one day of the park's operations.

### Provided Database

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkins', 'attractions', and 'sequences'. The information contained in each of these tables is listed below:

`checkins`:
    - Description: check-in data for all visitors for the day in the park. The data includes two types of check-ins, inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type
`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carussel, etc.
    - Fields: AttractionID, Name, Region, Category, type
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
The database is named 'dinofunworld.db' and is located in the 'readonly' directory of the Jupyter Notebook environment. It can be accessed at 'readonly/dinofunworld.db'.

### Questions to Answer

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.

Question 1: What is the most popular attraction to visit in the park?
Question 2: What ride (note that not all attractions are rides) has the longest visit time?
Question 3: Which Fast Food offering has the fewest visitors?
Question 4: Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline.

#### Administrative Notes

This assignment will be graded by Coursera's grading system. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed.

In [1]:
import sqlite3
import pandas as pd
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
db_filename = 'dinofunworld.db'
connection = sqlite3. connect(db_filename) 
connection_cursor= connection.cursor()

In [2]:
# Graded Cell
# PartID:  NDnou
# Question 1: What is the most popular attraction to visit in the park?
# Notes: Your output should be the name of the attraction.
sql_query = "SELECT attraction, COUNT(*) as c FROM checkin GROUP BY attraction ORDER BY c desc;"
connection_cursor.execute(sql_query)
popular_attraction_id = connection_cursor.fetchall()[0][0]
connection_cursor.execute("SELECT Name FROM attraction where AttractionID = {};".format(popular_attraction_id))
popular_attraction = connection_cursor.fetchone()[0]
print(popular_attraction)

Atmosfear


In [3]:
# Graded Cell
# PartID: FXGHp
# Question 2: What ride (note that not all attractions are rides) has the longest average visit time?
# Notes: Your output should be the name of the ride.
checkin_df = pd.read_sql_query("SELECT * FROM checkin;", connection)
checkin_df['total_seconds'] = pd.to_timedelta(checkin_df['duration']).astype('timedelta64[s]')
attraction_df = pd.read_sql_query("SELECT * FROM attraction WHERE Category LIKE '%Ride%';", connection)
attraction_df = attraction_df[['AttractionID','Name']]
merge_df = checkin_df[['attraction','total_seconds']].merge(attraction_df,left_on=['attraction'],right_on=['AttractionID'])
mean_df = merge_df.groupby('Name', as_index=False)['total_seconds'].mean()
print(mean_df.iloc[mean_df.total_seconds.argmax()]['Name'])

TerrorSaur


In [4]:
# Graded Cell
# PartID: KALua
# Question 3: Which Fast Food offering in the park has the fewest visitors?
# Notes: Your output should be the name of the fast food offering.
checkin_df = pd.read_sql_query("SELECT * FROM checkin", connection)
attraction_df = pd.read_sql_query("SELECT * FROM attraction WHERE type LIKE '%Fast Food%'", connection)
merge_df = checkin_df[['visitorID','attraction']].merge(attraction_df[['Name','AttractionID']],left_on=['attraction'],right_on=['AttractionID'])
final_df = merge_df.groupby(['Name']).size().reset_index(name='counts')
fast_food_with_fewest_visitors = final_df.iloc[final_df.counts.argmin()]['Name']
print(fast_food_with_fewest_visitors)

Theresaur Food Stop


In [5]:
# Graded Cell
# PartID: B0LUP
# Question 4: Compute the Skyline of number of visits and visit time for the park's ride and 
#  report the rides that appear in the Skyline. 
# Notes: Remember that in this case, higher visits is better and lower visit times are better. 
#  Your output should be formatted as an array listing the names of the rides in the Skyline.
df = pd.read_sql("SELECT attraction, COUNT(*) as visit_count FROM checkin WHERE attraction IN (SELECT AttractionID FROM attraction where LOWER(Category) LIKE '%ride%') GROUP BY attraction;",connection)
highest_visited_count = 0
best_ride_time = math.inf
best_both_attraction_id_list = [0,0,math.inf]
for attraction_id in df['attraction'].tolist():
    duration_df = pd.read_sql("SELECT duration FROM checkin where attraction="+str(attraction_id)+";",connection)
    duration_df['duration'] = pd.to_timedelta(duration_df['duration']).astype('timedelta64[s]')
    mean_duration = duration_df.mean()[0] 
    count = df.loc[df['attraction'] == attraction_id, 'visit_count'].iloc[0]
    if count > highest_visited_count:
        highest_visited_attraction_id = attraction_id
        highest_visited_count = count
    if  mean_duration < best_ride_time:
        best_ride_time_attraction_id = attraction_id
        best_ride_time = mean_duration
    if count > best_both_attraction_id_list[1] and mean_duration < best_both_attraction_id_list[2]:
        best_both_attraction_id_list = (attraction_id, count, mean_duration)
connection_cursor.execute("SELECT Name FROM attraction WHERE AttractionID="+str(highest_visited_attraction_id)+" OR AttractionID="+str(best_ride_time_attraction_id)+" OR AttractionID="+str(best_both_attraction_id_list[0])+";")
ride_names = connection_cursor.fetchall()
ride_name_list = [ride_name[0] for ride_name in ride_names]
print(ride_name_list)

['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']
